In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
from scipy.optimize import minimize
from __future__ import division
import statsmodels.api as sm

In [2]:
df = pd.read_csv(r'C:\Users\xelin\OneDrive\Desktop\Graduate School\3rd trimester\supply chain + forecasting\final\original data.csv')

In [3]:
df.Date = df.Date.apply(lambda x: pd.Timestamp(x))

In [4]:
df.dtypes

Product_Code                object
Warehouse                   object
Product_Category            object
Date                datetime64[ns]
Order_Demand                 int64
dtype: object

In [5]:

df['Order_Demand'] = df['Order_Demand'].replace('(','')
df['Order_Demand'] = df['Order_Demand'].replace(')','')
df['Order_Demand'] = pd.to_numeric(df['Order_Demand'])
df = df[(df['Order_Demand']>0)]
perw = df.Date.dt.to_period("W")
perm = df.Date.dt.to_period("M")
#sort by producct and warehouse 

df['Product_Code'] = df['Product_Code'].str.replace(r'[^0-9]+', '')
df['Warehouse'] = df['Warehouse'].str.replace(r'Whse_', '')
df['Product_Category'] = df['Product_Category'].str.replace(r'[^0-9]+', '')


In [6]:
#perw = df.Date.dt.to_period("W")
#perm = df.Date.dt.to_period("M")
df_w = df.groupby(['Product_Code', perw]).agg({'Order_Demand':np.sum , 'Warehouse':pd.Series.mode, 'Product_Category':pd.Series.mode})
df_w.reset_index(inplace=True)
df_m = df.groupby(['Product_Code', perm]).agg({'Order_Demand':np.sum , 'Warehouse':pd.Series.mode, 'Product_Category':pd.Series.mode})
df_m.reset_index(inplace=True)



Expontial Smoothing with Weekly Data


In [7]:
testw = df_w[df_w.Product_Code == '0001'].reset_index()[['Order_Demand']]
testw

,Order_Demand
0,200
1,100
2,1300
3,6200
4,900
...,...
228,100
229,500
230,600
231,800


In [8]:
testw.Order_Demand.values

array([  200,   100,  1300,  6200,   900,  1100,  1200,  3000,   300,
        4500,  4200,  1100,  2500,  6000,  4500,   500,   900,   900,
        1000,  6100,  2400,   200,   500,  2000,  1000,  2900,   500,
        6100,  1000,   400,  2600,   200,  4000,  1200,  2600,  1300,
        2700,   500,  3500,  8000,  6800,  1200,  4300,  1200,   200,
        1400,   300,  1200,  4100,   300,   400,  2500,  3000,  1400,
        4700,   100,  3100,   800,  7700,  4100,   300,  3600,   600,
        2800,   300,  2000,   300,  1200,  4900,  2400,  2200,   200,
        2300,   300,  3600,  2800,  1000,  2300,   100,  1000,  1700,
         200,  3300,  1200,   400,   700,  3500,  6200,  2800,  2100,
        3800,  4500,  2700,  1600,  1600,  1100,  3100,  1300,   700,
         100,   300,  3700,   900,   800,  4900,   200,   100,  2000,
        5000,   700,   100,   600,  1500,  4100,  2600,  4000,  3200,
        1000,  1400,  2200,  5000,  1100,  1500,  2000,  1800,  4200,
         300,  2100,

In [9]:


def y(alpha, x):   #exponential smoothers
    y = np.zeros(len(x), float)
    y[0] = x[0]
    for i in range(1, len(x)):
        y[i] = x[i-1]*alpha + y[i-1]*(1-alpha)
    return y

def mae(alpha, x):
    diff = np.abs(y(alpha, x) - x)
    return np.mean(diff)

def mae_2(alpha, x): #funct to optimize:mape
    err = 0
    for j in range(1, len(x)):
        err += np.abs(y(alpha, x[:j+1])[j] - x[j])
    return err

guess = .5
resultw = minimize(mae, guess, (testw.Order_Demand.values,), bounds=[(0,1)], method='SLSQP')

In [10]:
def aux_the_minimizator(group):
    guess = .5
    resultw = minimize(mae, guess, (group.values,), bounds=[(0,1)], method='SLSQP')
    return resultw.fun, resultw.x

def the_minimizator(df):
    coll_d = []
    for i in np.unique(df.Product_Code):
        df_aux = df[df.Product_Code == i].reset_index()[['Order_Demand']]
        mae, alpha = aux_the_minimizator(df_aux.Order_Demand)
        coll_d.append((i, mae, alpha, df_aux.shape[0],))
    return pd.DataFrame(coll_d , columns = ['Product_Code', 'ES_Mae', 'ES_alpha', 'n_obs'])

df_esw = pd.DataFrame()
df_esw = the_minimizator(df_w)


In [11]:
#test to check number of obs and unique product codes 
dftest = df_esw.copy()
dftest.Product_Code = dftest.Product_Code.astype(int)
dftest.sort_values(by = 'Product_Code')

,Product_Code,ES_Mae,ES_alpha,n_obs
0,1,1462.350866,[0.01904815828334811],233
1,2,34489.425860,[0.00850031761556717],205
2,3,389.868041,[0.007650230470661936],182
3,4,605.652725,[0.06909970010025925],161
4,5,332.313937,[0.017834612545123255],47
...,...,...,...,...
2155,2168,160.722538,[0.009935121411422679],248
2156,2169,79.035143,[0.004668220640386448],254
2157,2170,9.749735,[0.0168833919615704],137
2158,2171,6.629479,[0.0023891270156377858],239


In [12]:
#merge with week dataset
df_esw['Product_Code'] = df_esw['Product_Code'].apply(lambda x: '{0:0>3}'.format(x))
df_w['Product_Code'] = df_w['Product_Code'].apply(lambda x: '{0:0>3}'.format(x))
df_w_ES_merged = df_w.merge(df_esw, how='left', left_on='Product_Code', right_on='Product_Code')

In [13]:
#adding predictions back into main df
df_w_ES_merged['Product_Code'] = df_w_ES_merged['Product_Code'].astype('int')
df_w_ES_merged['shift'] = df_w_ES_merged.assign(dif = (df_w_ES_merged.Product_Code.diff() == 0).astype(int)).groupby(['Product_Code']).Order_Demand.shift(1)
df_w_ES_merged['ES_pred'] = df_w_ES_merged['shift'] + df_w_ES_merged['ES_alpha']*(df_w_ES_merged['Order_Demand'] - df_w_ES_merged['shift'])
df_w_ES_merged.drop(['shift'], axis=1, inplace=True)

Expontial Smoothing with Monthly Data

In [14]:
testm = df_m[df_m.Product_Code == '0001'].reset_index()[['Order_Demand']]
testm

,Order_Demand
0,300
1,9700
2,13000
3,14100
4,3300
...,...
56,17500
57,3300
58,2100
59,600


In [15]:
testm.Order_Demand.values

array([  300,  9700, 13000, 14100,  3300,  9200,  6400, 10100,  9300,
        6700, 16000,  7400,  6000,  6900,  8700, 12100,  7000,  3800,
        9700,  9000,  4400,  5800, 11400, 13200,  5900,  6200,  5000,
        5900,  7800,  6300,  9800, 10100, 10100,  8900, 10900, 12400,
        5000,  3500,  9100,  7200,  5800,  5400,  4000,  6500, 11400,
        9000,  2300,  4200,  2200,  2100,  6300,  9300,  3400, 16900,
        5300,  9300, 17500,  3300,  2100,   600,  1700], dtype=int64)

In [16]:


def y(alpha, x):   #exponential smoothers
    y = np.zeros(len(x), float)
    y[0] = x[0]
    for i in range(1, len(x)):
        y[i] = x[i-1]*alpha + y[i-1]*(1-alpha)
    return y

def mae(alpha, x):
    diff = np.abs(y(alpha, x) - x)
    return np.mean(diff)

def mae_2(alpha, x): #funct to optimize:mape
    err = 0
    for j in range(1, len(x)):
        err += np.abs(y(alpha, x[:j+1])[j] - x[j])
    return err

guess = .5
resultm = minimize(mae, guess, (testm.Order_Demand.values,), bounds=[(0,1)], method='SLSQP')

In [17]:
def aux_the_minimizator(group):
    guess = .5
    resultm = minimize(mae, guess, (group.values,), bounds=[(0,1)], method='SLSQP')
    return resultm.fun, resultm.x

def the_minimizator(df):
    coll_d = []
    for i in np.unique(df.Product_Code):
        df_aux = df[df.Product_Code == i].reset_index()[['Order_Demand']]
        mae, alpha = aux_the_minimizator(df_aux.Order_Demand)
        #es_pred = 
        coll_d.append((i, mae, alpha, df_aux.shape[0],))
    return pd.DataFrame(coll_d , columns = ['Product_Code', 'ES_Mae', 'ES_alpha', 'n_obs'])

df_esm = pd.DataFrame()
df_esm = the_minimizator(df_m)

In [18]:
#test to check number of obs and unique product codes 
dftest = df_esm.copy()
dftest.Product_Code = dftest.Product_Code.astype(int)
dftest.sort_values(by = 'Product_Code')

,Product_Code,ES_Mae,ES_alpha,n_obs
0,1,3463.915535,[0.5737163935276077],61
1,2,90638.770835,[0.4804919633178123],61
2,3,1007.039406,[0.07994563145007264],60
3,4,1061.422428,[0.11010917040467995],56
4,5,460.089742,[0.03402980347529575],34
...,...,...,...,...
2155,2168,338.306648,[0.16488624229569848],60
2156,2169,217.279145,[0.26304803749376066],61
2157,2170,19.805020,[0.34363680805513785],57
2158,2171,15.133333,[0.0],60


In [19]:
#merge with month dataset
df_m['Product_Code'] = df_m['Product_Code'].apply(lambda x: '{0:0>3}'.format(x))
df_m_ES_merged = df_m.merge(df_esm, how='left', left_on='Product_Code', right_on='Product_Code')

In [20]:
#adding predictions back into main df
df_m_ES_merged['Product_Code'] = df_m_ES_merged['Product_Code'].astype('int')
df_m_ES_merged['shift'] = df_m_ES_merged.assign(dif = (df_m_ES_merged.Product_Code.diff() == 0).astype(int)).groupby(['Product_Code']).Order_Demand.shift(1)
df_m_ES_merged['ES_pred'] = df_m_ES_merged['shift'] + df_m_ES_merged['ES_alpha']*(df_m_ES_merged['Order_Demand'] - df_m_ES_merged['shift'])
df_m_ES_merged.drop(['shift'], axis=1)


,Product_Code,Date,Order_Demand,Warehouse,Product_Category,ES_Mae,ES_alpha,n_obs,ES_pred
0,1,2011-12,300,J,005,3463.915535,[0.5737163935276077],61,[nan]
1,1,2012-01,9700,J,005,3463.915535,[0.5737163935276077],61,[5692.934099159513]
2,1,2012-02,13000,J,005,3463.915535,[0.5737163935276077],61,[11593.264098641106]
3,1,2012-03,14100,J,005,3463.915535,[0.5737163935276077],61,[13631.088032880369]
4,1,2012-04,3300,J,005,3463.915535,[0.5737163935276077],61,[7903.862949901836]
...,...,...,...,...,...,...,...,...,...
105519,2172,2016-07,1000,J,005,204.523788,[0.23469494817868694],30,[923.4694948178687]
105520,2172,2016-08,900,J,005,204.523788,[0.23469494817868694],30,[976.5305051821313]
105521,2172,2016-09,1200,J,005,204.523788,[0.23469494817868694],30,[970.4084844536061]
105522,2172,2016-10,900,J,005,204.523788,[0.23469494817868694],30,[1129.591515546394]


Regression on Weekly Data


In [21]:
df_w['Product_Code'] = df_w['Product_Code'].astype('int')
df_w['count'] = df_w.assign(dif = (df_w.Product_Code.diff() == 0).astype(int)).groupby(['Product_Code']).dif.apply(lambda x : x.cumsum().rank(method='dense'))

In [22]:
df_w['Product_Code'] = df_w['Product_Code'].astype('str')


In [23]:


def linear_est_lt(df, PC):
    
    assert isinstance(PC, str) == True    
    df_by_PC = df[df.loc[:,'Product_Code'] == PC][['count','Order_Demand']].reset_index(drop = True)

    try:
        assert df_by_PC.shape[0] > 1
    
        X = df_by_PC.drop(columns= ['Order_Demand'], axis = 1)
        X = sm.add_constant(X)
        y = df_by_PC.loc[:,'Order_Demand']

        mod = sm.OLS(y,X)
        res = mod.fit()


        #print(res.summary())
        est_param = res.params.to_dict()
        t = True, PC, res
        #print(supplier, est_param)
        return t
    except:
        #print(f"Not enough datapoints, only {df_by_supplier.shape[0]} for supplier {supplier}")
        t = False, PC, "Not Enough DataPoints"
        return t

def _build_single_db(df, PC):
    """
    This function builds a database for each supplier
    """
    create, pc, res = linear_est_lt(df, PC)
    if create:
        param = res.params.to_dict()
        tuple_all = pc, param['const'], param['count']
        
        db_prod = pd.DataFrame([tuple_all], columns = ['Product_Code', 'const', 'b_count'])
        return db_prod
    else:
        return False


In [24]:
def build_db(df):

    product_codes = df['Product_Code'].unique()
    final_db = _build_single_db(df, product_codes[0])  
    excluded = []
    for pc in product_codes[1:]:
        try:
            add_db = _build_single_db(df, pc)
        except:
            print(pc)
        if isinstance(add_db, bool):
            excluded.append(pc)
        else:
            final_db = pd.concat([final_db, _build_single_db(df, pc)])
    return excluded, final_db.reset_index(drop = True)

In [25]:
build_db(df_w)

(['465', '853', '1703', '2099'],
      Product_Code         const    b_count
 0               1   2337.512950  -3.244214
 1               2  37171.449067  55.688039
 2               3    402.993139   2.681514
 3               4    524.945652   3.050667
 4               5    332.654949   5.908881
 ...           ...           ...        ...
 2151         2168    210.800085  -0.112269
 2152         2169    118.217018  -0.197194
 2153         2170     12.013418   0.023396
 2154         2171      9.105341   0.000621
 2155         2172    161.556330  11.644113
 
 [2156 rows x 3 columns])

In [26]:
_, final = build_db(df_w)

In [27]:
df_reg_w = df_w.merge(final, how='left', left_on='Product_Code', right_on='Product_Code')

In [28]:
df_reg_w['forecast'] = df_reg_w['const'] + (df_reg_w['count'] * df_reg_w['b_count'])
df_reg_w['reg_error'] = abs(df_reg_w['forecast'] - df_reg_w['Order_Demand'])


In [29]:
df_reg_errorw = df_reg_w.groupby(['Product_Code']).agg({'reg_error':np.mean})
df_reg_errorw.reset_index(inplace=True)


In [30]:
df_w_ES_merged.dtypes

Product_Code                int32
Date                period[W-SUN]
Order_Demand                int64
Warehouse                  object
Product_Category           object
ES_Mae                    float64
ES_alpha                   object
n_obs                       int64
ES_pred                    object
dtype: object

In [31]:
#merge with weekly
df_w_ES_merged['Product_Code'] = df_w_ES_merged['Product_Code'].astype('object')
df_reg_errorw['Product_Code'] = df_reg_errorw['Product_Code'].apply(lambda x: '{0:0>4}'.format(x))
df_w_ES_merged['Product_Code'] = df_w_ES_merged['Product_Code'].apply(lambda x: '{0:0>4}'.format(x))
df_final_w = df_w_ES_merged.merge(df_reg_errorw, how='left', left_on='Product_Code', right_on='Product_Code')

In [32]:

df_final_w['reg_forecast'] = df_reg_w['forecast']

Regression with Montly data

In [33]:
df_m['Product_Code'] = df_m['Product_Code'].astype('int')
df_m['count'] = df_m.assign(dif = (df_m.Product_Code.diff() == 0).astype(int)).groupby(['Product_Code']).dif.apply(lambda x : x.cumsum().rank(method='dense'))

In [34]:
df_m['Product_Code'] = df_m['Product_Code'].astype('str')


In [35]:
build_db(df_m)

(['465', '853', '1703', '2099'],
      Product_Code          const     b_count
 0               1    9043.442623  -50.475939
 1               2  134132.786885  324.643046
 2               3    1085.141243   28.902473
 3               4     818.506494   49.162679
 4               5     374.331551   16.088617
 ...           ...            ...         ...
 2151         2168     883.299435   -2.287413
 2152         2169     488.818033   -3.266473
 2153         2170      30.823935    0.066567
 2154         2171      38.261582   -0.055571
 2155         2172      13.563218   36.329255
 
 [2156 rows x 3 columns])

In [36]:
_, final = build_db(df_m)

In [37]:
df_reg_m = df_m.merge(final, how='left', left_on='Product_Code', right_on='Product_Code')

In [38]:
df_reg_m['forecast'] = df_reg_m['const'] + (df_reg_m['count'] * df_reg_m['b_count'])
df_reg_m['reg_error'] = abs(df_reg_m['forecast'] - df_reg_m['Order_Demand'])

In [39]:
df_reg_errorm = df_reg_m.groupby(['Product_Code']).agg({'reg_error':np.mean})
df_reg_errorm.reset_index(inplace=True)


In [40]:
#merge with monthly
df_m_ES_merged['Product_Code'] = df_m_ES_merged['Product_Code'].astype('object')
df_reg_errorm['Product_Code'] = df_reg_errorm['Product_Code'].apply(lambda x: '{0:0>4}'.format(x))
df_m_ES_merged['Product_Code'] = df_m_ES_merged['Product_Code'].apply(lambda x: '{0:0>4}'.format(x))
df_final_m = df_m_ES_merged.merge(df_reg_errorm, how='left', left_on='Product_Code', right_on='Product_Code')

In [41]:
df_final_m['reg_forecast'] = df_reg_m['forecast']

Moving Average Weekly

In [42]:
df_final_w.sort_values(by=['Product_Code','Date'], inplace = True)

In [43]:
df_wma = pd.DataFrame()
df_wma['Order_Demand'] = df_final_w['Order_Demand']
df_wma['Product_Code'] = df_final_w['Product_Code']
df_wma['Date'] = df_final_w['Date']
df_wma['rolling_mean'] = df_wma.set_index('Date').groupby('Product_Code', sort=False)['Order_Demand'].rolling(3,freq='d').mean().round(2).to_numpy()
df_wma.dropna(inplace = True)



In [44]:
df_wma['roll_error'] = abs(df_wma['rolling_mean'] - df_wma['Order_Demand'])
df_wma2 = df_wma.groupby(['Product_Code']).agg({'roll_error':np.mean})
df_wma2.reset_index(inplace=True)


In [45]:
df_final_w['rolling_mean'] = df_wma['rolling_mean']

In [46]:
df_final_w = df_final_w.merge(df_wma2, how='left', left_on='Product_Code', right_on='Product_Code')


Moving Average Monthly

In [47]:
df_final_m.sort_values(by=['Product_Code','Date'], inplace = True)

In [48]:
df_mma = pd.DataFrame()
df_mma['Product_Code'] = df_final_m['Product_Code']
df_mma['Order_Demand'] = df_final_m['Order_Demand']
df_mma['Date'] = df_final_m['Date']

df_mma['rolling_mean'] = df_mma.set_index('Date').groupby('Product_Code', sort=False)['Order_Demand'].rolling(3,freq='d').mean().round(2).to_numpy()
df_mma.dropna(inplace = True)

In [49]:
df_mma['roll_error'] = abs(df_mma['rolling_mean'] - df_mma['Order_Demand'])
df_mma2 = df_mma.groupby(['Product_Code']).agg({'roll_error':np.mean})
df_mma2.reset_index(inplace=True)


In [50]:
df_final_m['rolling_mean'] = df_mma['rolling_mean']

In [51]:
df_final_m = df_final_m.merge(df_mma2, how='left', left_on='Product_Code', right_on='Product_Code')


Fixup

In [52]:

df_final_w['ES_pred'] = df_final_w['ES_pred'].astype('str')
df_final_m['ES_pred'] = df_final_m['ES_pred'].astype('str')
df_final_w['Date'] = df_final_w['Date'].astype('str')
df_final_m['Date'] = df_final_m['Date'].astype('str')

In [53]:

df_final_w['ES_pred'] = df_final_w['ES_pred'].str.strip('[]')
df_final_m['ES_pred'] = df_final_m['ES_pred'].str.strip('[]')



In [54]:
df_final_w['Date'] = [x.split('/')[0] for x in df_final_w['Date']]

In [68]:

Lowest_es = df_final_m.groupby(['Product_Code']).ES_MAPE.mean().sort_values()


Conversion to CSV

In [63]:

df_final_w.to_csv(r'C:\Users\xelin\OneDrive\Desktop\Graduate School\3rd trimester\supply chain + forecasting\final\final_weeklyII.csv')
df_final_m.to_csv(r'C:\Users\xelin\OneDrive\Desktop\Graduate School\3rd trimester\supply chain + forecasting\final\final_monthlyII.csv')

Code takes 11:52 minutes to run